In [ ]:
# Initialisation cell
import numpy as np
import pandas as pd
import seaborn as sns
import math
from matplotlib import pyplot as plt
from sklearn import preprocessing
from sklearn.metrics import mean_squared_error as mse
from sklearn.model_selection import train_test_split as tts

In this prokect I have made use of the [video game sales](https://www.kaggle.com/gregorut/videogamesales) dataset from kaggle which contains the sales rankings of videos games that have sold over 100 000 copies in several market regions. The columns from the dataset that will be extracted as input features are the platform the game was released on, the genre the game is classified under adn the pulisher of the videogame.

I will be using a neural network with a variety of hyperparameters, including various activation functions, namely: linear, sigmoid, tanh and ReLU, to see how they affect the accuracy of prediction. Comparisions will be made between three different numbers of hidden layes, namely: one, three and five, to see how these variations affect the prediction.

## Data Wrangling
I will be using a pandas dataframe to import the data that is in CSV format. Only the columns related to the name of the game, its release platform, the genre, and the publisher will be extracted as input features. The column containing the global sales (Global_Sales, in millions) will also be extracted as the response variable. 

Once the data is loaded into a pandas data frame, any observations containg a column with a null value are culled from the data frame. This may introduce bias by removing samples from the dataset, however only a few observations needed to be removed so the effect should be minor.

The Platform, Genre and Publisher columns contain object/string values, meaning these need to be encoded into integers to allow for their use in the neural network. Simple label encoding is used to give each unique string value in a column an associated integer value.

In [ ]:
df = pd.read_csv('vgsales.csv',index_col=0)

df = df.loc[:,['Name','Platform','Genre','Publisher','Global_Sales']]
df = df.dropna(axis='index') # removing all rows containg null values (might cause biasing)
df = df.drop_duplicates() # remove any duplicate entries in the dataset

# Converting object types to category types
df.Platform = df.Platform.astype('category')
df.Genre = df.Genre.astype('category')
df.Publisher = df.Publisher.astype('category')

# Label encoding for all columns containing string values
df['Platform_cat'] = df.Platform.cat.codes
df['Publisher_cat'] = df.Publisher.cat.codes
df['Genre_cat'] = df.Genre.cat.codes
df = df.reset_index()

df.head()

## Data Visualisation
Below is a pair plot which can be used as early indicator of any correlation between attributes of the data. It is clear that looking at the distribution of the Global_Sales column, there are outliers skewing the data at the right tail, meaning these need to be removed in order to obtain more representative data that will allow the neural net to be more generalizable.

In [ ]:
# Plotting distribution of sales
plt.plot(df.Global_Sales)
plt.show()

In [ ]:
# Removing outliers in data
df = df[df.Global_Sales.between(df.Global_Sales.quantile(.15), df.Global_Sales.quantile(.85))]
plt.plot(df.Global_Sales)
plt.show()

In [ ]:
# EDA to find correlations in data
print(df.corr()) # correlation matrix
# Correlation Visualization
cols = ['Platform_cat', 'Genre_cat', 'Publisher_cat', 'Global_Sales']
pp = sns.pairplot(df[cols], height=1.5, aspect=1.5,plot_kws=dict(edgecolor="k", linewidth=0.5),diag_kind="kde", diag_kws=dict(shade=True))
fig = pp.fig 
fig.subplots_adjust(top=0.93, wspace=0.3)
t = fig.suptitle('Video game sales attributes', fontsize=14)

In [ ]:
# Splitting test and training data
X = df.loc[:,['Platform_cat', 'Genre_cat', 'Publisher_cat']]
Y = df.loc[:,"Global_Sales"]
X_train,X_test,Y_train,Y_test = tts(X.values, Y.values, test_size = 0.3) 

## Neural Network Implementations
The neural network will be implement with varying hyperparameters. Initially a single hidden layer will be used along with and input layer that has three input units and one output unit.  

In [ ]:
# Activation functions

# Sigmoid function
def sig(z): 
    return 1/(1+np.exp(-z))
# Linear function
def lin(z):
    return z
# tanh function
def tanh(z):
    return np.tanh(z)
# ReLU
def relu(z):
    return np.maximum(0,z)

In [ ]:
# Random initialization for weights
def rand_weights(dim_layers):
    weights = {}
    np.random.seed(25) # ensure reproducibility
    
    for i in range(1,len(dim_layers)):
        weights[i] = np.random.randn(dim_layers[i], dim_layers[i-1])/100

    return weights

### Foward propagation using different activation functions

In [ ]:
# Forward propagation using sigmoid

def for_prop_sig(x, weights):
    l = len(weights)
    a = {}
    z = {}

    z[1] = np.dot(weights[1], x) + 1
    a[1] = sig(z[1])

    for i in range(2, l+1):
        z[i] = np.dot(weights[i], a[i-1]) + 1
        if i==l:
            a[i] = z[i]
        else:
            a[i] = sig(z[i])
    
    return a

In [ ]:
# Forward propagation using linear

def for_prop_lin(x, weights):
    l = len(weights)
    a = {}
    z = {}
    
    z[1] = np.dot(weights[1], x) + 1
    a[1] = sig(z[1])

    for i in range(2, l+1):
        z[i] = np.dot(weights[i], a[i-1]) + 1
        if i==l:
            a[i] = z[i]
        else:
            a[i] = lin(z[i])
    
    return a

In [ ]:
# Forward propagation using tanh

def for_prop_tanh(x, weights):
    l = len(weights)
    a = {}
    z = {}
    
    z[1] = np.dot(weights[1], x) + 1
    a[1] = sig(z[1])

    for i in range(2, l+1):
        z[i] = np.dot(weights[i], a[i-1]) + 1
        if i==l:
            a[i] = z[i]
        else:
            a[i] = tanh(z[i])
    
    return a

In [ ]:
# Forward propagation using relu

def for_prop_relu(x, weights):
    l = len(weights)
    a = {}
    z = {}
    
    z[1] = np.dot(weights[1], x) + 1
    a[1] = sig(z[1])

    for i in range(2, l+1):
        z[i] = np.dot(weights[i], a[i-1]) + 1
        if i==l:
            a[i] = z[i]
        else:
            a[i] = relu(z[i])
    
    return a

In [ ]:
# Cost calculation

def cost_calc(a,y):
    l = len(a)

    hyp = a[l]
    cost = 1/(2*len(y)) * np.sum(np.square(hyp - y))
    
    return cost

In [ ]:
# Backward propagation

def back_prop(weights,a,x,y):
    l = len(weights)
    n = len(y)
    deltas = {}
  
    for i in range(l,0,-1):
        if i==l:
            da = 1/n*(a[i] - y)
            dz = da
        else:
            da = np.dot(weights[i+1].T, dz)
            dz = np.multiply(da, a[i])
        if i==1:
            deltas[i] = (1/n)*np.dot(dz, x.T)
        else:
            deltas[i] = (1/n)*np.dot(dz,a[i-1].T)
    
    return deltas

In [ ]:
# Update weights based on errors

def new_weights(weights,deltas,alpha):
    l = len(weights)

    n_weights = {}
    
    for i in range(1,l+1):
        n_weights[i] = weights[i] - alpha*deltas[i]
    
    return n_weights

### Training using different activation functions

In [ ]:
# Training of model using sigmoid activation functions

def train_sig(x, y, dim_layers, iterations, alpha):
    weights = rand_weights(dim_layers)
    iter_costs = []

    for i in range(iterations):
        a = for_prop_sig(x.T, weights)
        cost = cost_calc(a, y.T)
        deltas = back_prop(weights,a,x.T,y.T)
        weights = new_weights(weights,deltas,alpha)
        iter_costs.append(cost)
    
    return weights,iter_costs

In [ ]:
# Training of model using linear activation functions

def train_lin(x, y, dim_layers, iterations, alpha):
    weights = rand_weights(dim_layers)
    iter_costs = []

    for i in range(iterations):
        a = for_prop_lin(x.T, weights)
        cost = cost_calc(a, y.T)
        deltas = back_prop(weights,a,x.T,y.T)
        weights = new_weights(weights,deltas,alpha)
        iter_costs.append(cost)
    
    return weights,iter_costs

In [ ]:
# Training of model using tanh activation functions

def train_tanh(x, y, dim_layers, iterations, alpha):
    weights = rand_weights(dim_layers)
    iter_costs = []

    for i in range(iterations):
        a = for_prop_tanh(x.T, weights)
        cost = cost_calc(a, y.T)
        deltas = back_prop(weights,a,x.T,y.T)
        weights = new_weights(weights,deltas,alpha)
        iter_costs.append(cost)
    
    return weights,iter_costs

In [ ]:
# Training of model using ReLU activation functions

def train_relu(x, y, dim_layers, iterations, alpha):
    weights = rand_weights(dim_layers)
    iter_costs = []

    for i in range(iterations):
        a = for_prop_relu(x.T, weights)
        cost = cost_calc(a, y.T)
        deltas = back_prop(weights,a,x.T,y.T)
        weights = new_weights(weights,deltas,alpha)
        iter_costs.append(cost)
    
    return weights,iter_costs

### Mean square error calculations for various activation function predictions

In [ ]:
# Calculate accuracy of sigmoid activation function

def sig_err(x_train, x_test, y_train, y_test, weights,dim_layers):
    
    a_test = for_prop_sig(x_test.T, weights)
    a_train = for_prop_sig(x_train.T, weights)

    test_err = mse(y_test, a_test[len(dim_layers)-1].T)
    train_err = mse(y_train, a_train[len(dim_layers)-1].T)
    
    return test_err, train_err

In [ ]:
# Calculate accuracy of linear activation function

def lin_err(x_train, x_test, y_train, y_test, weights,dim_layers):

    a_test = for_prop_lin(x_test.T, weights)
    a_train = for_prop_lin(x_train.T, weights)

    test_err = mse(y_test, a_test[len(dim_layers)-1].T)
    train_err = mse(y_train, a_train[len(dim_layers)-1].T)
    
    return test_err, train_err

In [ ]:
# Calculate accuracy of tanh activation function

def tanh_err(x_train, x_test, y_train, y_test, weights,dim_layers):

    a_test = for_prop_tanh(x_test.T, weights)
    a_train = for_prop_tanh(x_train.T, weights)

    test_err = mse(y_test, a_test[len(dim_layers)-1].T)
    train_err = mse(y_train, a_train[len(dim_layers)-1].T)
    
    return test_err, train_err

In [ ]:
# Calculate accuracy of relu activation function

def relu_err(x_train, x_test, y_train, y_test, weights,dim_layers):

    a_test = for_prop_relu(x_test.T, weights)
    a_train = for_prop_relu(x_train.T, weights)

    test_err = mse(y_test, a_test[len(dim_layers)-1].T)
    train_err = mse(y_train, a_train[len(dim_layers)-1].T)
    
    return test_err, train_err

In [ ]:
# Model paramenters
iterations = 1000
alpha = 0.05

# Varoius neural net hidden layer dimensions
dim_layers1 = [3,20,1]
dim_layers2 = [3,10,10,1]
dim_layers3 = [3,5,5,5,1]
dim_layers4 = [3,10,10,10,1]

## Model Test Using Sigmoid Activation Function
The following contains the training for a neural network using the sigmoid activation function and various dimensions for the hidden layers.

In [ ]:
# Calculating weights and costs over iterations for various hidden layer dimensions

weights_sig1,costs_sig1 = train_sig(X_train, Y_train, dim_layers1, iterations, alpha)
weights_sig2,costs_sig2 = train_sig(X_train, Y_train, dim_layers2, iterations, alpha)
weights_sig3,costs_sig3 = train_sig(X_train, Y_train, dim_layers3, iterations, alpha)
weights_sig4,costs_sig4 = train_sig(X_train, Y_train, dim_layers4, iterations, alpha)

# Error calculations for various hidden layer dimensions

test_err_sig1, train_err_sig1 = sig_err(X_train, X_test, Y_train, Y_test, weights_sig1,dim_layers1)
test_err_sig2, train_err_sig2 = sig_err(X_train, X_test, Y_train, Y_test, weights_sig2,dim_layers2)
test_err_sig3, train_err_sig3 = sig_err(X_train, X_test, Y_train, Y_test, weights_sig3,dim_layers3)
test_err_sig4, train_err_sig4 = sig_err(X_train, X_test, Y_train, Y_test, weights_sig4,dim_layers4)

In [ ]:
# Plots of iteration costs for various hidden layer dimensions

iter = np.arange(1,1001)
figure = plt.figure(figsize=(7,6))
ax = figure.add_subplot(1,1,1)
ax.plot(iter,costs_sig1,'r',label='1 hidden layer of 20 unit')
ax.plot(iter,costs_sig2,'g',label='2 hidden layers of 10 units')
ax.plot(iter,costs_sig3,'b',label='3 hidden layers of 5 units')
ax.plot(iter,costs_sig4,'orange',label='3 hidden layers of 10 units')
ax.set_xlabel('Iterations');ax.set_ylabel('Costs');ax.set_title('Cost reduction using sigmoid function');ax.legend()
plt.show()

From the above plot, it can be seen that a network architecture that contains 1 hidden layer of 20 units has costs that converges fastest over the 1000 iterations. The configurations using 2 hidden layers starts off with a lower cost than using 3 hidden layers, but converges at a similar rate to using 3 hidden layers.

In [ ]:
# Plots of errors for various hidden layer dimensions

figure = plt.figure(figsize=(7,6))
ax = figure.add_subplot(1,1,1)
ax.scatter(x=test_err_sig1,y=train_err_sig1,c='r',label='1 hidden layer of 20 unit')
ax.scatter(x=test_err_sig2,y=train_err_sig2,c='g',label='2 hidden layers of 10 units')
ax.scatter(x=test_err_sig3,y=train_err_sig3,c='b',label='3 hidden layers of 5 units')
ax.scatter(x=test_err_sig4,y=train_err_sig4,c='orange',label='3 hidden layers of 10 units')
ax.set_xlabel('Train error');ax.set_ylabel('Test error');ax.set_title('Errors using sigmoid function');ax.legend()
plt.show()

From the above plot, it can be seen that using 1 hidden layer consisting of 20 units each gives the least train and testing error, followed by the use of 2 hidden layer with 10 units. This is consistent with the convergence rates from the previous plot.

## Model Test Using Linear Activation Function
The following contains the training for a neural network using the linear activation function and various dimensions for the hidden layers.

In [ ]:
# Calculating weights and costs over iterations for various hidden layer dimensions

weights_lin1,costs_lin1 = train_lin(X_train, Y_train, dim_layers1, iterations, alpha)
weights_lin2,costs_lin2 = train_lin(X_train, Y_train, dim_layers2, iterations, alpha)
weights_lin3,costs_lin3 = train_lin(X_train, Y_train, dim_layers3, iterations, alpha)
weights_lin4,costs_lin4 = train_lin(X_train, Y_train, dim_layers4, iterations, alpha)

# Error calculations for various hidden layer dimensions

test_err_lin1, train_err_lin1 = lin_err(X_train, X_test, Y_train, Y_test, weights_lin1,dim_layers1)
test_err_lin2, train_err_lin2 = lin_err(X_train, X_test, Y_train, Y_test, weights_lin2,dim_layers2)
test_err_lin3, train_err_lin3 = lin_err(X_train, X_test, Y_train, Y_test, weights_lin3,dim_layers3)
test_err_lin4, train_err_lin4 = lin_err(X_train, X_test, Y_train, Y_test, weights_lin4,dim_layers4)

In [ ]:
# Plots of iteration costs for various hidden layer dimensions

iter = np.arange(1,1001)
figure = plt.figure(figsize=(7,6))
ax = figure.add_subplot(1,1,1)
ax.plot(iter,costs_lin1,'r',label='1 hidden layer of 20 unit')
ax.plot(iter,costs_lin2,'g',label='2 hidden layers of 10 units')
ax.plot(iter,costs_lin3,'b',label='3 hidden layers of 5 units')
ax.plot(iter,costs_lin4,'orange',label='3 hidden layers of 10 units')
ax.set_xlabel('Iterations');ax.set_ylabel('Costs');ax.set_title('Cost reduction using linear function');ax.legend()
plt.show()

In the abover graph, the use of a single hidden layer with 20 units produced the fastest convergence of the cost, followed by the use of 2 hidden layers with 10 units in each layer, which starts off at a lower cost value but converges slower than using 1 hidden layer. Using a 3 hidden layer configuration consisting of 5 units in each layer produced the worst results. This shows that the linear function favours the use of a large number of units in a layer over the use of large number of layers with few units in each layer. 

In [ ]:
# Plots of errors for various hidden layer dimensions

figure = plt.figure(figsize=(7,6))
ax = figure.add_subplot(1,1,1)
ax.scatter(x=test_err_lin1,y=train_err_lin1,c='r',label='1 hidden layer of 20 unit')
ax.scatter(x=test_err_lin2,y=train_err_lin2,c='g',label='2 hidden layers of 10 units')
ax.scatter(x=test_err_lin3,y=train_err_lin3,c='b',label='3 hidden layers of 5 units')
ax.scatter(x=test_err_lin4,y=train_err_lin4,c='orange',label='3 hidden layers of 10 units')
ax.set_xlabel('Train error');ax.set_ylabel('Test error');ax.set_title('Errors using linear function');ax.legend()
plt.show()

The accuracy when using a linear function is optimal in this project when using 2 hidden layers with 10 units in each layer, followed by the use of a 1 hidden layer with 10 units in each layer. In this case, the most accurate and fastest converging configuration is the same, namely: 1 hidden layer with 20 units each.

## Model Test Using tanh Activation Function
The following contains the training for a neural network using the linear activation function and various dimensions for the hidden layers.

In [ ]:
# Calculating weights and costs over iterations for various hidden layer dimensions

weights_tanh1,costs_tanh1 = train_tanh(X_train, Y_train, dim_layers1, iterations, alpha)
weights_tanh2,costs_tanh2 = train_tanh(X_train, Y_train, dim_layers2, iterations, alpha)
weights_tanh3,costs_tanh3 = train_tanh(X_train, Y_train, dim_layers3, iterations, alpha)
weights_tanh4,costs_tanh4 = train_tanh(X_train, Y_train, dim_layers4, iterations, alpha)

# Error calculations for various hidden layer dimensions

test_err_tanh1, train_err_tanh1 = tanh_err(X_train, X_test, Y_train, Y_test, weights_tanh1,dim_layers1)
test_err_tanh2, train_err_tanh2 = tanh_err(X_train, X_test, Y_train, Y_test, weights_tanh2,dim_layers2)
test_err_tanh3, train_err_tanh3 = tanh_err(X_train, X_test, Y_train, Y_test, weights_tanh3,dim_layers3)
test_err_tanh4, train_err_tanh4 = tanh_err(X_train, X_test, Y_train, Y_test, weights_tanh4,dim_layers4)

In [ ]:
# Plots of iteration costs for various hidden layer dimensions

iter = np.arange(1,1001)
figure = plt.figure(figsize=(7,6))
ax = figure.add_subplot(1,1,1)
ax.plot(iter,costs_tanh1,'r',label='1 hidden layer of 20 unit')
ax.plot(iter,costs_tanh2,'g',label='2 hidden layers of 10 units')
ax.plot(iter,costs_tanh3,'b',label='3 hidden layers of 5 units')
ax.plot(iter,costs_tanh4,'orange',label='3 hidden layers of 10 units')
ax.set_xlabel('Iterations');ax.set_ylabel('Costs');ax.set_title('Cost reduction using tanh function');ax.legend()
plt.show()

When using a tanh activation function, the 1 hidden layer configuration with 20 units converges fastest, followed by the configuration using 3 hidden layers with 10 units. The configurations using 2 and 3 hidden layers start off with lower cost values than using a single hidden layer, but converge slower.

In [ ]:
# Plots of errors for various hidden layer dimensions

figure = plt.figure(figsize=(7,6))
ax = figure.add_subplot(1,1,1)
ax.scatter(x=test_err_tanh1,y=train_err_tanh1,c='r',label='1 hidden layer of 20 unit')
ax.scatter(x=test_err_tanh2,y=train_err_tanh2,c='g',label='2 hidden layers of 10 units')
ax.scatter(x=test_err_tanh3,y=train_err_tanh3,c='b',label='3 hidden layers of 5 units')
ax.scatter(x=test_err_tanh4,y=train_err_tanh4,c='orange',label='3 hidden layers of 10 units')
ax.set_xlabel('Train error');ax.set_ylabel('Test error');ax.set_title('Errors using tanh function');ax.legend()
plt.show()

From the above plot, using 1 hidden layer leads to the least error, followed by using 2 hidden layers with 10 units each. This corresponds with convergence rates plotted before and shows than tanh generealizes better with more units used.

## Model Test Using ReLU Activation Function
The following contains the training for a neural network using the ReLU activation function and various dimensions for the hidden layers.

In [ ]:
# Calculating weights and costs over iterations for various hidden layer dimensions

weights_relu1,costs_relu1 = train_relu(X_train, Y_train, dim_layers1, iterations, alpha)
weights_relu2,costs_relu2 = train_relu(X_train, Y_train, dim_layers2, iterations, alpha)
weights_relu3,costs_relu3 = train_relu(X_train, Y_train, dim_layers3, iterations, alpha)
weights_relu4,costs_relu4 = train_relu(X_train, Y_train, dim_layers4, iterations, alpha)

# Error calculations for various hidden layer dimensions

test_err_relu1, train_err_relu1 = relu_err(X_train, X_test, Y_train, Y_test, weights_relu1,dim_layers1)
test_err_relu2, train_err_relu2 = relu_err(X_train, X_test, Y_train, Y_test, weights_relu2,dim_layers2)
test_err_relu3, train_err_relu3 = relu_err(X_train, X_test, Y_train, Y_test, weights_relu3,dim_layers3)
test_err_relu4, train_err_relu4 = relu_err(X_train, X_test, Y_train, Y_test, weights_relu4,dim_layers4)

In [ ]:
# Plots of iteration costs for various hidden layer dimensions

iter = np.arange(1,1001)
figure = plt.figure(figsize=(7,6))
ax = figure.add_subplot(1,1,1)
ax.plot(iter,costs_relu1,'r',label='1 hidden layer of 20 unit')
ax.plot(iter,costs_relu2,'g',label='2 hidden layers of 10 units')
ax.plot(iter,costs_relu3,'b',label='3 hidden layers of 5 units')
ax.plot(iter,costs_relu4,'orange',label='3 hidden layers of 10 units')
ax.set_xlabel('Iterations');ax.set_ylabel('Costs');ax.set_title('Cost reduction using ReLU function');ax.legend()
plt.show()

The above plot shows that the 1 hidden layer configuration with 20 units coverges fastest but starts off with the error higher than the configuration using 2 hidden layers, which allows the 2 hidden layer configuration to reach a lower cost value in 1000 iterations.

In [ ]:
# Plots of errors for various hidden layer dimensions

figure = plt.figure(figsize=(7,6))
ax = figure.add_subplot(1,1,1)
ax.scatter(x=test_err_relu1,y=train_err_relu1,c='r',label='1 hidden layer of 20 unit')
ax.scatter(x=test_err_relu2,y=train_err_relu2,c='g',label='2 hidden layers of 10 units')
ax.scatter(x=test_err_relu3,y=train_err_relu3,c='b',label='3 hidden layers of 5 units')
ax.scatter(x=test_err_relu4,y=train_err_relu4,c='orange',label='3 hidden layers of 10 units')
ax.set_xlabel('Train error');ax.set_ylabel('Test error');ax.set_title('Errors using relu function');ax.legend()
plt.show()

After 1000 iterations, the 2 hidden layer configuration using the ReLU activation function has the least error even though it converges slower than using 1 hidden layer. This is due to the 2 hidden layer configuration starting off at a lower cost value than the 1 hidden layer configuration. 

## Conclusion

From the above implementations, the trend seems to be that using a higher number of units increases accuracy and decrerases convergence rate across the board. All the activation functions produce good results from just using a single hidden layer, this can be explained by the simplicity of the data used and the fact that only a single output feature is being predicted by the model. 

The various activation functions coverge to different accuracy values after 1000 iterations, the most accurate out of the four used was a close call between ReLU and Linear activations functions. This can be due to the linear relationships present within the data as shown by the Data Wrangling section of the project, indicating that the choice of activation fucntion is depenedent on the problem type.